In [1]:
!pip install --user stable_baselines3 gym

In [2]:
import os
from gym import Env
from gym.spaces import Box
import numpy as np
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env

In [3]:
np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.4f}'.format

In [4]:
M_0 = 5
m_0 = 0
STARTING_CAPITAL = 15
C_0 = 1_020_000
PROFIT_MARGIN = 0.2
AOV = 300

In [5]:
order_freq_week = pd.read_csv(os.path.join('..', 'input', 'customer-growth-strategy', 'order_frequency_data.csv'), index_col=0)
order_dist = order_freq_week.groupby('order_frequency_per_week', as_index=False)[['user_id']].count()

In [6]:
total_users_in_sample, total_orders_in_sample = order_freq_week.user_id.count(), np.dot(order_dist.order_frequency_per_week, order_dist.user_id)
total_users_in_sample, total_orders_in_sample

In [7]:
order_dist['weights'] = order_dist['user_id'] / order_dist['user_id'].sum()
order_dist['order_freq_per_day'] = order_dist['order_frequency_per_week'] / 7
weights = order_dist['weights']
order_freq_per_day = order_dist['order_freq_per_day']

In [8]:
def probability_deliver(m_i):
    return 0.8 * (1.25 - np.exp(-1.0 * m_i/5))

In [9]:
def growth_after_spends(m_i, M_i):
    return (38/40) + ((0.45 + probability_deliver(m_i)) ** 0.3 + (M_i/(M_0 + 1)) ** 0.05) / 40

In [10]:
def estimated_orders(customer_base_size):
    return np.sum(weights * customer_base_size * order_freq_per_day)

estimated_orders(C_0)

In [11]:
def customer_base_growth(C_i_minus_1, m_i, M_i):
    return C_i_minus_1 * growth_after_spends(m_i, M_i)
customer_base_growth(C_0, m_0, M_0), customer_base_growth(C_0, 0, M_0) - C_0

In [12]:
def profit_on_day_i(customer_base_size):
    return estimated_orders(customer_base_size) * PROFIT_MARGIN * AOV / 10 ** 6
profit_on_day_i(C_0)

In [13]:
def obj_fun(x, n):
    m, M = x[:n].copy(), x[n:].copy()

    capital_available_day_i_start = [STARTING_CAPITAL]
    C_i_minus_1 = [None]
    m_i_arr = [m_0]
    p = [probability_deliver(m_0)]
    M_i_arr = [M_0]
    growth = [None]
    C_i = [C_0]
    total_orders = [estimated_orders(C_0)]
    total_expenditure = [m_0 + M_0]
    profit = [profit_on_day_i(C_0)]
    capital_available_day_i_end = [STARTING_CAPITAL - m_0 - M_0 + profit_on_day_i(C_0)]
    
    for i, (m_i, M_i) in enumerate(zip(m, M)):
        capital_available_day_i_start.append(capital_available_day_i_end[-1])
        C_i_minus_1.append(C_i[-1])
        m_i_arr.append(m_i)
        p.append(probability_deliver(m_i))
        M_i_arr.append(M_i)
        growth.append(growth_after_spends(m_i, M_i))
        C_i.append(customer_base_growth(C_i_minus_1[-1], m_i, M_i))
        total_orders.append(estimated_orders(C_i[-1]))
        total_expenditure.append(m_i_arr[-1] + M_i_arr[-1])
        profit.append(profit_on_day_i(C_i[-1]))
        capital_available_day_i_end.append(capital_available_day_i_start[-1] - m_i_arr[-1] - M_i_arr[-1] + profit[-1])

    return C_i[-1] / C_0

In [14]:
def create_df(capital_available_day_i_start, C_i_minus_1, m_i_arr, p, M_i_arr, growth, C_i, total_orders, profit, total_expenditure, capital_available_day_i_end):
    data = {
      "capital_available_day_i_start": capital_available_day_i_start,
      "C_i_minus_1": C_i_minus_1,
      "m_i_arr": m_i_arr,
      "p": p,
      "M_i_arr": M_i_arr,
      "growth": growth,
      "C_i": C_i,
      "total_orders": total_orders,
      "total_expenditure": total_expenditure,
      "profit": profit,
      "capital_available_day_i_end": capital_available_day_i_end,
    }
    return pd.DataFrame(data)

def create_investment_schedule(x, n):
    m, M = x[:n].copy(), x[n:].copy()

    capital_available_day_i_start = [STARTING_CAPITAL]
    C_i_minus_1 = [None]
    m_i_arr = [m_0]
    p = [probability_deliver(m_0)]
    M_i_arr = [M_0]
    growth = [None]
    C_i = [C_0]
    total_orders = [estimated_orders(C_0)]
    total_expenditure = [m_0 + M_0]
    profit = [profit_on_day_i(C_0)]
    capital_available_day_i_end = [STARTING_CAPITAL - m_0 - M_0 + profit_on_day_i(C_0)]
    
    for i, (m_i, M_i) in enumerate(zip(m, M)):
        capital_available_day_i_start.append(capital_available_day_i_end[-1])
        C_i_minus_1.append(C_i[-1])
        m_i_arr.append(m_i)
        p.append(probability_deliver(m_i))
        M_i_arr.append(M_i)
        growth.append(growth_after_spends(m_i, M_i))
        C_i.append(customer_base_growth(C_i_minus_1[-1], m_i, M_i))
        total_orders.append(estimated_orders(C_i[-1]))
        total_expenditure.append(m_i_arr[-1] + M_i_arr[-1])
        profit.append(profit_on_day_i(C_i[-1]))
        capital_available_day_i_end.append(capital_available_day_i_start[-1] - m_i_arr[-1] - M_i_arr[-1] + profit[-1])

    return create_df(capital_available_day_i_start, C_i_minus_1, m_i_arr, p, M_i_arr, growth, C_i, total_orders, profit, total_expenditure, capital_available_day_i_end)

In [15]:
class CustomerGrowthEnv(Env):
    def __init__(self):
        self.num_customers = np.array([C_0], dtype=np.float32)
        self.observation_space = Box(low=np.array([500_000]), high=np.array([3e6]), shape=(1,), dtype=np.float32)
        self.time_steps = 30
        self.action_space = Box(low=np.array([0.01, 0.01]), high=np.array([100.,100.]), shape=(2,), dtype=np.float32)
        self.available_capital = np.array([10.], dtype=np.float32)
        self.weights = weights
        self.order_freq_per_day = order_freq_per_day
        self.PROFIT_MARGIN = 0.2
        self.AOV = 300
        self.M_0 = 5

    def probability_deliver(self, m_i):
        return np.array([0.8 * (1.25 - np.exp(-1.0 * m_i/5))])

    def growth_after_spends(self, m_i, M_i):
        return np.array((38/40) + ((0.45 + self.probability_deliver(m_i)) ** 0.3 + (M_i/(self.M_0 + 1)) ** 0.05) / 40, dtype=np.float32)

    def estimated_orders(self, customer_base_size):
        return np.sum(self.weights * customer_base_size * self.order_freq_per_day)

    def customer_base_growth(self, C_i_minus_1, m_i, M_i):
        return np.array(C_i_minus_1 * self.growth_after_spends(m_i, M_i), dtype=np.float32)

    def profit_on_day_i(self, customer_base_size):
        return np.array([self.estimated_orders(customer_base_size) * self.PROFIT_MARGIN * self.AOV / 10 ** 6], dtype=np.float32)

    def step(self, action):
        if self.time_steps == 30:
            action = np.array([0.0, 5.0])
        
        total_expenditure = action[0] + action[1]
        
        if total_expenditure <= self.available_capital:
            # C_prev = self.num_customers
            self.num_customers = self.customer_base_growth(self.num_customers, action[0], action[1])
            profit = self.profit_on_day_i(self.num_customers)
            self.available_capital = self.available_capital - total_expenditure + profit

            # reward = float((self.num_customers / C_prev) - 1)
            reward = float((self.num_customers / C_0) - 1)
            self.time_steps -= 1 
            
            done = False
            if self.time_steps <= 0: done = True
            
            info = {}

            return self.num_customers, reward, done, info
        else:
            return self.num_customers, float(-0.25), False, {}

    def render(self, *args, **kwargs):
        pass
    
    def reset(self):
        self.num_customers = np.array([C_0], dtype=np.float32)
        self.time_steps = 30 
        return self.num_customers

In [16]:
env=CustomerGrowthEnv()

In [17]:
env.observation_space.sample()

In [18]:
env.reset()

In [19]:
check_env(env, warn=True)

In [20]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [21]:
env.close()

In [22]:
MIN_LR, MAX_LR = 0.0003, 0.001
def learning_rate_policy(progress_remaining):
    return MIN_LR + (MAX_LR - MIN_LR) * progress_remaining

In [23]:
model = PPO("MlpPolicy", env, verbose=1, learning_rate=learning_rate_policy)

In [24]:
model.learn(total_timesteps=10_00_000)

In [25]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [26]:
obs = env.reset()
m, M = [], []
for i in range(30):
    action, _states = model.predict(obs)
    m.append(action[0])
    M.append(action[1])
    obs, rewards, dones, info = env.step(action)

In [27]:
obj_fun(m + M, 30)

In [28]:
results = create_investment_schedule(m + M, 30)
results.to_csv('./investment_schedule.csv', index=False)

In [29]:
results